This is code to accompany the paper "Supersingular curves via the Shimura--Taniyama method" by Jeremy Booher and Rachel Pries.  The goal is to enumerate all covers of the projective line with bounded genus which are supersingular modulo p for certain congruences class of primes.  This is inspired by an old result of Ekedahl who constructed superspecial curves of genus 4 and 5 in certain characteristics using this approach.  However, this approach has not been systematically explored until now.  See the paper for notation and references.

In [1]:
def _partial_enumeration(genus_bound,d,e,f):
    """Enumerate possible (d,e,f),r,s given (d,e,f).  Used in enumerate_covers"""
    new_stuff = []
    r = 1
    while r * e * f - (1 + f/d + e/d) <= 2 * genus_bound -2:
        if r * e * f - (1 + f/d + e/d) <=0 : #won't have genus > 1
            r = r+1
            continue
        s = 1
        while  s * ( r * e * f - (1 + f/d + e/d)) <= 2 * genus_bound - 2:
            #is genus an integer?  is s| rd satisfied? are d,e,f pairwise relatively prime
            g = s * ( r * e * f - (1 + f/d + e/d)) /2 + 1
            if g in ZZ and r * d /s in ZZ and gcd(d,e) == 1 and gcd(d,f) == 1 and gcd(e,f) ==1: 
                new_stuff.append([[d,e,f],r,s])
            s = s+1
        r = r+1

    return new_stuff


def enumerate_covers(genus_bound):
    """Find all abelian covers of the projective line with three points of ramification and 1<genus <= genus_bound.
    Note that requiring genus>1 is fine and removes an infinite family of boring examples of covers
    
    Returns a list of (d,e,f), s ,r using Ekedahl's notation.
    
    Note: d,e,f are e_{i,j} in our notation"""

    candidates = []
    #remember that d<=e<=f by convention
    f = 1
    
    while f <= 2*genus_bound + 2:
        e = 1
        
        while e <= f:
            done_e = true
            d = 1 
            while d <= e and e * f - (1 + f/d + e/d) <= 2 * genus_bound-2:
                new_stuff = _partial_enumeration(genus_bound,d,e,f)
                candidates.extend(new_stuff)
                d = d +1
                
            e = e +1        
        
        f = f +1
    return candidates

In [2]:
def convert_cover(triple):
    """Convert to the (a,b,c,s) notation from the (d,e,f), s ,r (both used by Ekedahl).
    a,b,c are the orders of the generators of inertia, s is the index of last inertia in G"""
    d,e,f = expand( triple[0])
    r = triple[1]
    s = triple[2]

    a = r * d * e
    b = r * d * f
    c = r * e * f
    genus = s * ( r * e * f - (1 + f/d + e/d)) /2 + 1
    
    return [a,b,c,s],genus

In [3]:
def cyclic_inertial_generators(a,b,c,s):
    """Assuming the cover has s=1, find possible generators for inertia with given (a,b,c,s).
    Prune the list to account for isomorphic covers"""

    assert(s == 1)
    
    inert_gens = [] #inertial generators to return
    isomorphic_gens =[] #cases isomorphic to ones in inert_gens
    
    # cyclic Galois group is Z/(c)
    order = c 
    G = Integers(order)

    #up to automorphism, may as well assume generator for last inertia group is 1
    third = G(1)
  
    #as generators of inertia sum to 0, look at possible values for the first and see which are valid:
    #first has order a and second has order b

    for g in G:
        if g.order() == a and (-third - g ).order() == b:
            first = g
            second = -third - g

            data = [first.lift(),second.lift(),third.lift()]

            if data not in isomorphic_gens:
                #it's new
                inert_gens.append(data)
                isomorphic_gens.append(data)

                #if some of a,b,c are equal, may permuate and scale to get an isomorphic cover.  Only write down one of them
                #in particular, if a=b!= c could swap first and second inertial elements.
                #if c = b != a, could swap and rescale
                #if c= b= = a, could use S_3 permutation and rescale
                if a == b and b != c:
                    isomorphic_gens.append([second.lift(),first.lift(),third.lift()])
                elif c == b and b != a:
                    scaling = second^(-1)
                    isomorphic_gens.append([(first*scaling).lift(),scaling.lift(),1])
                elif c == b and b == a:
                    isomorphic_gens.append([second.lift(),first.lift(),third.lift()])
                    isomorphic_gens.append([first^(-1) * second,first^(-1),1])
                    isomorphic_gens.append([first^(-1),first^(-1) * second,1])
                    isomorphic_gens.append([second^(-1),second^(-1) * first,1])
                    isomorphic_gens.append([second^(-1) * first,second^(-1),1])
                    
    return inert_gens,order

def alternate_inertial_generators(a,b,c,s):
    """Assuming the cover is a product Z/(c) \times Z/(s) with s|c,
    find possible generators for inertia (up to automorphism) with given (a,b,c,s)"""

    inert_gens = []
    # Galois group is Z/(c) \times Z/(s) 
    order = c * s
    G = AdditiveAbelianGroup([c,s])

    #up to automorphism, may as well assume generator for last inertia group is (1,0) as it has order c
    third = G([1,0])

    #the other two generators cannot lie in <(1,0)> as they together generate the whole group.

    #as generators of inertia sum to 0, look at possible values for the first and see which are valid:
    #first has order a and second has order b

    for g in G:
        if g.order() == a and (-third - g ).order() == b:
            #inertia groups must also generate whole group
            if G.submodule([g,third]).order() == c * s: 
                inert_gens.append([g,-third-g,third])
                
    return inert_gens,G

In [4]:
def present_by_genus(min,max):
    """Create dictionaries of all covers where min <= genus <= max, indexed by genus"""
    data  = enumerate_covers(max)

    collected_covers = {} #(a,b,c,s)
    for n in range(min,max+1):
        collected_covers[n] = []

    for triple in data:
        data,genus = convert_cover(triple)
        if min <= genus and genus <= max:
            collected_covers[genus].append(data)

    return collected_covers

Example: We can enumerate all covers with genus up to 10, then display those with genus 5

In [5]:
biglist = enumerate_covers(10)

In [6]:
print("(a,b,c,s) Ramification Info for Cyclic Cover")

for triple in biglist:
    data,genus = convert_cover(triple)
    if genus == 5:
        print(data)

(a,b,c,s) Ramification Info for Cyclic Cover
[11, 11, 11, 1]
[4, 8, 8, 2]
[6, 12, 12, 1]
[3, 15, 15, 1]
[2, 12, 12, 2]
[2, 20, 20, 1]
[2, 11, 22, 1]


We need to decompose the covers where s is not 1 in order by hand.

In [7]:
print("Examples with Z/cZ times Z/sZ")
mydict=present_by_genus(4,10)

for genus in range(4,11):
    print("Genus: ",genus)
    data = mydict[genus]
    for tup in data:
        if tup[3] > 1:
            print(tup)
            print(alternate_inertial_generators(*tup))
            print()
    print("---------------")

Examples with Z/cZ times Z/sZ
Genus:  4
[6, 6, 6, 2]
([[(1, 1), (4, 1), (1, 0)], [(4, 1), (1, 1), (1, 0)]], Additive abelian group isomorphic to Z/6 + Z/2)

[3, 6, 6, 3]
([[(2, 1), (3, 2), (1, 0)], [(0, 1), (5, 2), (1, 0)], [(4, 1), (1, 2), (1, 0)], [(4, 2), (1, 1), (1, 0)], [(2, 2), (3, 1), (1, 0)], [(0, 2), (5, 1), (1, 0)]], Additive abelian group isomorphic to Z/6 + Z/3)

[2, 10, 10, 2]
([[(5, 1), (4, 1), (1, 0)], [(0, 1), (9, 1), (1, 0)]], Additive abelian group isomorphic to Z/10 + Z/2)

---------------
Genus:  5
[4, 8, 8, 2]
([[(2, 1), (5, 1), (1, 0)], [(6, 1), (1, 1), (1, 0)]], Additive abelian group isomorphic to Z/8 + Z/2)

[2, 12, 12, 2]
([[(6, 1), (5, 1), (1, 0)], [(0, 1), (11, 1), (1, 0)]], Additive abelian group isomorphic to Z/12 + Z/2)

---------------
Genus:  6
[5, 5, 5, 5]
([[(0, 1), (4, 4), (1, 0)], [(0, 2), (4, 3), (1, 0)], [(0, 3), (4, 2), (1, 0)], [(0, 4), (4, 1), (1, 0)], [(1, 1), (3, 4), (1, 0)], [(1, 2), (3, 3), (1, 0)], [(1, 3), (3, 2), (1, 0)], [(1, 4), (3, 1)

Now compute the Newton-Polygon in various characteristics using the Shimura-Taniyama method.

In [8]:
#auxiliary functions
def compute_fj(j,a1,a2,a3,m):
    return -1 + frac(-1 * j * a1/m) + frac(-1 * j * a2/m) + frac(-1 * j * a3/m)

def compute_dj(j,m):
    return ((Integers(m))(j)).order() #it is additive order

In [9]:
#there is probably a way to get the orbits using built-in methods, but it's also easy to do directly
def find_orbits(p,m):
    """Find the orbits of the multiplication by p map on Z/mZ - {0}"""
    G = Integers(m)
    used_elements = {G(0)} 
    orbits = {G(0)}
    orbits.remove(G(0)) #don't want the trivial orbit

    for g in G:
        if g not in used_elements:
            new_orbit = [g]
            current = p * g
            while current != g:
                new_orbit.append(current)
                used_elements.add(current)
                current = p * current
            orbits.add(tuple(new_orbit))
    return orbits

def compute_slopes(p,a1,a2,a3,m):
    """Computes the slopes, returing a list of the G_{c,d} appearing in the decomposition"""
    orbits = find_orbits(p,m)
    slopes = []

    for orbit in orbits:
        j = orbit[0]
        dj = compute_dj(j,m)
        if a1 % dj != 0 and a2 % dj != 0 and a3 % dj != 0:
            ones = sum([compute_fj(n.lift(),a1,a2,a3,m) for n in orbit])
            slopes.append((ones,len(orbit)-ones))
    return slopes 

def is_supersingular(p,a1,a2,a3,m):
    """Is the cover of degree m with inertial generators a1,a2,a3 supersingular in characteristic p?  Note depends only on p modulo m"""
    slopes = compute_slopes(p,a1,a2,a3,m)

    for chunk in slopes:
        if chunk[1] != 0 and chunk[0]/chunk[1] != 1: #equivalent to slope 1/2
            return false

    return true

In [10]:
def find_supersingular(a1,a2,a3,m):
    """Find all congruence class of primes modulo m (with gcd(p,m)=1) such that the cover is supersingular"""
    congruence_classes = []
    for p in range(m):
        if gcd(p,m) == 1:
            if is_supersingular(p,a1,a2,a3,m):
                congruence_classes.append(p)
    return congruence_classes

In [11]:
def process_ss_primes(primes,m):
    """Reformat the list of supersingular primes to look nicer"""
    #check if it's all QNR's, also change format
    R = Integers(m)
    QR = { x^2 for x in R if gcd(x,m) ==1}
    QNR = { x for x in R if gcd(x,m)==1 and x not in QR}

    not1 = { x for x in R if gcd(x,m)==1 and x != R(1)}

    if {R(x) for x in primes} == QNR:
        return "\\equiv QNR"
    elif {R(x) for x in primes} == not1:
        return "\\not \\equiv 1"
    else:    
        return '\\equiv ' + ', '.join([str(p) for p in primes])

def table_all_supersingular(min,max):
    """Create a table of all supersingular 3-pointed covers with min <= genus <= max with s =1"""

    collected_covers = present_by_genus(min,max)

    data_by_genus = []

    for g in range(min,max+1):
        rows = [['z','a','supersingular primes']]
        for data in collected_covers[g]:
            if data[3] == 1: #will be cyclic
                z = data # [a,b,c,s]
                possible_gens,m = cyclic_inertial_generators(*data)

                first_entry = [data[2],data[1],data[0],data[3]]
                for alist in possible_gens:
                    rows.append([first_entry, (alist[2],alist[1],alist[0]), "$p "+ process_ss_primes(find_supersingular(*alist,m),m) + " \\pmod{" + str(m) + "}$"])
                    first_entry = ""
                
        data_by_genus.append(rows)

    return data_by_genus

In [12]:
example_data=table_all_supersingular(5,10)

In [13]:
for row in example_data:
    print('\\begin{table}[h]')
    print(latex(table(row,frame=True)))
    print('\\caption{Genus $$}')
    print('\\end{table}')
    print()

\begin{table}[h]
\begin{tabular}{|l|l|l|} \hline
z & a & supersingular primes \\ \hline
$\left[11, 11, 11, 1\right]$ & $\left(1, 9, 1\right)$ & $p \equiv QNR \pmod{11}$ \\ \hline
 & $\left(1, 8, 2\right)$ & $p \equiv QNR \pmod{11}$ \\ \hline
$\left[12, 12, 6, 1\right]$ & $\left(1, 1, 10\right)$ & $p \equiv 11 \pmod{12}$ \\ \hline
$\left[15, 15, 3, 1\right]$ & $\left(1, 4, 10\right)$ & $p \equiv 11, 14 \pmod{15}$ \\ \hline
$\left[20, 20, 2, 1\right]$ & $\left(1, 9, 10\right)$ & $p \equiv 11, 19 \pmod{20}$ \\ \hline
$\left[22, 11, 2, 1\right]$ & $\left(1, 10, 11\right)$ & $p \equiv QNR \pmod{22}$ \\ \hline
\end{tabular}
\caption{Genus $$}
\end{table}

\begin{table}[h]
\begin{tabular}{|l|l|l|} \hline
z & a & supersingular primes \\ \hline
$\left[13, 13, 13, 1\right]$ & $\left(1, 11, 1\right)$ & $p \equiv 2, 4, 5, 6, 7, 8, 10, 11, 12 \pmod{13}$ \\ \hline
 & $\left(1, 10, 2\right)$ & $p \equiv 2, 4, 5, 6, 7, 8, 10, 11, 12 \pmod{13}$ \\ \hline
 & $\left(1, 9, 3\right)$ & $p \equiv 2, 4, 5, 6

In [14]:
#decomposed covers with $s>1$ by hand

non_cyclic_congruences = {}
non_cyclic_moduli = {}

#genus 5
congruence_5 = ([7],[11])
modulus_5 = (8,12)

non_cyclic_congruences[5] = congruence_5
non_cyclic_moduli[5] = modulus_5

#genus 6

congruence_6 = ( find_supersingular(1,1,3,5), find_supersingular(1,6,7,14) )
modulus_6 = (5,14)

non_cyclic_congruences[6] = congruence_6
non_cyclic_moduli[6] = modulus_6

#genus 7
#multiple_ss([[1,1,1,3], [1,3,5,9],[1,2,6,9]]) is =2 mod 3
#multiple_ss([[1,2,9,12], [1,3,8,12]]) is =11 mod 12
#multiple_ss([[1,7,8,16], [1,3,4,8],[1,1,2,4]]) is =15 mod 16
congruence_7 = ([2],[11],[15])
modulus_7 = (3,12,16)

non_cyclic_congruences[7] = congruence_7
non_cyclic_moduli[7] = modulus_7

#genus 8
#multiple_ss([[1,1,8,10], [1,3,6,10],[1,1,3,5]]) gives = 3,7,9 mod 10
#multiple_ss([[1,8,9,18]]) gives =5,11,17 mod 18

congruence_8 = ( [3,7,9],[5,11,17])
modulus_8 = (10,18)

non_cyclic_congruences[8] = congruence_8
non_cyclic_moduli[8] = modulus_8

#genus 9
#two were seen to not yield anything beyond the cyclic case, so can be ignored
#multiple_ss([[1, 9, 10,20],[1, 4, 5,10]]) gives = 19 mod 20

congruence_9 = ( [19], )
modulus_9 = (20,)

non_cyclic_congruences[9] = congruence_9
non_cyclic_moduli[9] = modulus_9

#genus 10
#multiple_ss([[1, 1, 7, 9],[1, 1,1,3]]) gives = 2 mod 3
#multiple_ss([[1, 3, 8, 12],[1, 4,7,12]]) gives =11 mod 12
#find_supersingular(1,10,11,22) gives [7, 13, 17, 19, 21] mod 22

congruence_10 = ( [2], [11],[7, 13, 17, 19, 21] )
modulus_10 = (3,12,22)

non_cyclic_congruences[10] = congruence_10
non_cyclic_moduli[10] = modulus_10

The data above came from the decompositions of corvers from the second half our paper

In [15]:
#find primes that make multiple cyclic covers supersingular at once

#covers represented as [a1,a2,a3 , m]
def multiple_ss(covers):
    congruence_classes = [find_supersingular(*cover) for cover in covers]
    moduli = [cover[3] for cover in covers]

    return congruence_classes,moduli

Finally we compute the density of primes for which there is a supersingular 3-pointed cover of each genus.  

In [19]:
def _compute_moduli(congruence_list,modulus_list):
    m = lcm(modulus_list)
    moduli = [0 for m in range(m)]

    for i in range(len(congruence_list)):
        mi = modulus_list[i]
        for a in congruence_list[i]:
            assert(a < mi)
            for n in range(m/mi):
                moduli[a+n * mi] = 1
    return [n for n in range(m) if moduli[n] ==1 and gcd(n,m)==1],m       

def compute_densities(min,max):
    """Compute the density of primes for which we have supersingular 3-pointed covers with min <= genus <= max.
    Relies on non_cyclic_congruences[g] being non-empty (i.e. we worked out what happens when s != 1).
    If empty, just analyzes the cyclic examples."""

    collected_covers = present_by_genus(min,max)
    data_by_genus = []

    for g in range(min,max+1):
        if g in non_cyclic_congruences.keys():
            modulus_list = list(non_cyclic_moduli[g])
            congruence_list = list(non_cyclic_congruences[g])
        else:
            print("Warning: Did not analyze covers where s != 1 for genus ",g)
            print("Density Only Reflects Covers where s=1")
            modulus_list = []
            congruence_list = []

        for data in collected_covers[g]:
            if data[3] == 1: #will be cyclic
                z = data # [a,b,c,s]
                possible_gens,m = cyclic_inertial_generators(*data)

                for alist in possible_gens:
                    congruence_list.append(find_supersingular(*alist,m))
                    modulus_list.append(m)
               
        combined,common_modulus = _compute_moduli(congruence_list,modulus_list)
        #print(combined)
        #print(common_modulus)
        ratio = len(combined)/euler_phi(common_modulus)
        data_by_genus.append(ratio)

    return data_by_genus

In [20]:
compute_densities(5,10)

[25/32, 507/512, 3/4, 1023/1024, 15/16, 31/32]

In [49]:
print([1.0 * x for x in compute_densities(5,10) ])

[0.781250000000000, 0.990234375000000, 0.750000000000000, 0.999023437500000, 0.937500000000000, 0.968750000000000]
